In [2]:
import classla
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ipywidgets import IntProgress
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [9]:
tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/sloberta")

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrejkronovsek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# download standard models for Slovenian
classla.download('sl')
nlp = classla.Pipeline('sl', processors='tokenize,ner,pos,lemma,depparse')

2022-05-24 22:45:05 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2022-05-24 22:45:06 INFO: File exists: /home/andrejkronovsek/classla_resources/sl/pos/standard.pt.
2022-05-24 22:45:06 INFO: File exists: /home/andrejkronovsek/classla_resources/sl/lemma/standard.pt.
2022-05-24 22:45:06 INFO: File exists: /home/andrejkronovsek/classla_resources/sl/depparse/standard.pt.
2022-05-24 22:45:06 INFO: File exists: /home/andrejkronovsek/classla_resources/sl/ner/standard.pt.
2022-05-24 22:45:06 INFO: File exists: /home/andrejkronovsek/classla_resources/sl/pretrain/standard.pt.
2022-05-24 22:45:06 INFO: Finished downloading models and saved to /home/andrejkronovsek/classla_resources.
2022-05-24 22:45:06 INFO: Loading these models for language: sl (Slovenian):
| Processor

In [11]:
stopwords = set(stopwords.words('slovene'))
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for p in punc:
    stopwords.add(p)

In [10]:
with open("../../data/data.json", "r") as f:
    data = json.loads(f.read())

In [7]:
content = []
for article in data:
    content.append(article["title"] + " " + article["subtitle"] + " " + article["headline"])

In [8]:
len(content[:20])

20

In [18]:
lemma_all = []
for doc in content[:20]:
    lemma = [i for i in nlp(doc).get("lemma") if i not in stopwords]
    lemma_all.append(lemma)

In [19]:
with open("lemma.json", "r") as f:
    lemma_all = json.loads(f.read())

In [21]:
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(lemma_all)]

In [11]:
max_epochs = 100
vec_size = 5
alpha = 0.025

model = Doc2Vec(vector_size=vec_size, alpha=alpha,  min_alpha=0.00025, min_count=1, dm=1)
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_02.model")
print("Model Saved")

NameError: name 'tagged_data' is not defined

In [15]:
model= Doc2Vec.load("d2v.model")

# to find most similar doc using tags
similar_doc = model.dv.distances("1")
print(similar_doc)

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.dv['100'])

[0.17422265 0.         1.236819   ... 1.0871044  0.94702226 1.0810812 ]
[ -2.2657537   7.6629596   8.755312   -1.1420728 -14.300156 ]


In [16]:
similar_doc = model.dv.distances("1")

In [17]:
titles = []
for article in data:
    titles.append(article["title"])

In [18]:
print(titles[100])
print("--------------------------")
for id, similarity in similar_doc:
    print(similarity, titles[int(id)])

Suvereni zmagi ZDA in Kanade
--------------------------


TypeError: cannot unpack non-iterable numpy.float32 object

In [19]:
vectors = []
for i in range(len(titles)):
    vectors.append(model.dv[str(i)])
np.array(vectors)

array([[ -5.9046116,  13.179378 ,   5.3790016,  -2.11712  ,   4.9323764],
       [ -1.3162087,   4.827221 ,   4.7891307,  -1.3558507,   7.129795 ],
       [  4.1281786,  10.8353615,  -5.4564576,  -7.8038425, -10.494016 ],
       ...,
       [ 16.904793 ,   3.2110507,  -3.0636005,  -4.5447364,  -0.0730581],
       [ -2.4866803,  14.419546 ,   6.791834 ,   1.7108026, -12.912988 ],
       [ 10.281287 ,  -7.1956286,   7.37295  ,  -8.873397 ,  -1.8259819]],
      dtype=float32)

In [20]:
pairwise_similarities=cosine_similarity(vectors)

In [21]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {titles[doc_id]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {titles[ix]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

In [22]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: WHO na Kitajsko pošilja strokovnjake, ki bodo iskali izvor covida-19


Similar Documents:


Document: Upad okužb na Novi Zelandiji, oblasti pravijo, da zaprtje deluje; Kosovo bo uničilo cepivo
Cosine Similarity : 0.9990981817245483


Document: WHO: Naslednje žarišče okužbe bi lahko bila Afrika
Cosine Similarity : 0.9988510608673096


Document: Od novembra bo v DR Kongo na voljo novo cepivo proti eboli
Cosine Similarity : 0.9985811710357666


Document: V Afriki potrdili okoli štiri milijone okužb, prihaja cepivo
Cosine Similarity : 0.998277485370636


Document: Bocvana: Večina primerov omikrona asimptomatičnih. Guterres kritičen do omejitve potovanj.
Cosine Similarity : 0.996977686882019


Document: V ZDA samo v januarju zaradi covida-19 umrlo več ljudi kot zaradi gripe v zadnjih treh letih
Cosine Similarity : 0.9964132308959961


Document: ZN: Lakota v svetu se je lani že tretje leto zapored povečala
Cosine Similarity : 0.9963365197181702


Document: Guterres: Svet na katastr

In [77]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(vectors)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

fig = plt.figure(figsize = (50, 50), facecolor="white", dpi=200)
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

ax.scatter(principalDf['principal component 1'], principalDf['principal component 2'])

In [61]:
data_df = pd.DataFrame.from_dict(data)
data_df["datetime_published"] = pd.to_datetime(data_df['datetime_published'], format='%Y-%m-%d %H:%M:%S')

In [4]:
content = []
for article in data:
    content.append(article["title"])

In [16]:
lemma_all = []
for doc in content[:100]:
    lemma = [i for i in nlp(doc).get("lemma") if i not in stopwords]
    lemma_all.append(lemma)

In [17]:
with open("lemma.json", "r") as f:
    lemma_all = json.loads(f.read())

In [87]:
similar_ix=np.argsort(pairwise_similarities[0])[::-1]

In [89]:
similar_ix[:10]

array([   0, 6568, 3010, 4897, 3709, 3343, 6913, 7539, 5848, 3785])

In [25]:
data_df = pd.DataFrame.from_dict(data)
data_df["datetime_published"] = pd.to_datetime(data_df['datetime_published'], format='%Y-%m-%d %H:%M:%S')

pairwise_similarities=cosine_similarity(vectors)

for idx, article in enumerate(data):
       date = datetime.datetime.strptime(article["datetime_published"], "%Y-%m-%d %H:%M:%S")
       window_start = date - datetime.timedelta(days=7)
       window_end = date + datetime.timedelta(days=7)

       data_in_window = data_df[data_df["datetime_published"] > window_start]
       data_in_window = data_in_window[data_in_window["datetime_published"] < window_end]

       similar_ix=np.argsort(pairwise_similarities[idx])[::-1]
       for i, row in data_in_window.iterrows():
              if i != idx:
                     if i in similar_ix[:2]:
                            print(pairwise_similarities[idx][i])
                            print(titles[idx])
                            print(titles[i])
                            print()

0.9949083
Dževad Karahasan: Vonj po strahu
Dela Katarine in Lojzeta Spacala, ki razkrivajo sobivanje s Krasom

0.9985121
Voznik traktorja po sobotni nesreči umrl v bolnišnici
Koroški gasilci že od nedelje zjutraj gasijo tleče komunalne odpadke na Holmcu

0.997246
Olimpijcem v Pekingu odsvetovano govoriti o človekovih pravicah
Žalna slovesnost za "leva v službi evropske demokracije" Davida Sassolija

0.99871624
Koalicija se bo do konca mandata osredotočila na epidemijo, zakon o dohodnini in energetske bone
Gospodarstvo za sprejetje zakona o dohodnini, SDS naklonjen referendumu v času volitev

0.9977806
Riiber do dvojčka na Holmenkollnu
Pedersen končal devetmesečno sušo, Roglič še naprej v odličnem položaju

0.9974179
Domžale po Olimpiji zdaj z 2:0 slavile še v Kopru 
Jordan navdušeno pozdravil Dončića, ki z Jokićem nadaljuje obračun z žogicami

0.9993431
Evropski parlamentarci kritizirali Hojsa zaradi izjave članic EU-ja o Afganistanu
DZ gosti medparlamentarno konferenco o skupni zunanj

KeyboardInterrupt: 

In [65]:
data_df = pd.DataFrame.from_dict(data)
data_df["datetime_published"] = pd.to_datetime(data_df['datetime_published'], format='%Y-%m-%d %H:%M:%S')

for idx, article in enumerate(data):
       date = datetime.datetime.strptime(article["datetime_published"], "%Y-%m-%d %H:%M:%S")
       window_start = date - datetime.timedelta(days=20)
       window_end = date + datetime.timedelta(days=20)

       data_in_window = data_df[data_df["datetime_published"] > window_start]
       data_in_window = data_in_window[data_in_window["datetime_published"] < window_end]

       possible_title_matches = []
       possible_title_matches.append(" ".join(lemma_all[idx]))
       for i, row in data_in_window.iterrows():
              if i != idx:
                     possible_title_matches.append(" ".join(lemma_all[i]))

       tfidf = TfidfVectorizer().fit_transform(possible_title_matches)
       pairwise_similarity = tfidf * tfidf.T

       similarity_array = pairwise_similarity.toarray()[1:,0]

       if len(similarity_array) > 3:
              idx_max = np.argpartition(similarity_array, -5)[-5:]
              for i in idx_max:
                     if similarity_array[i] > 0.7:
                            most_similar_aricle = data_in_window.iloc[[i]]
                            print(article["title"])
                            print("----------------------------")
                            print(similarity_array[i], most_similar_aricle.iloc[0]["title"])
                            print()

Odbor za delo: Pomanjkanje delovne sile bi lahko reševali z upokojenci
----------------------------
0.7227985894621086 Bi načrtovana nadgradnja hotela Piran skazila vedute mesta?

Red Tree Heritage: Naravno okolje najbolj učinkovito varuje lokalno prebivalstvo
----------------------------
0.7011540487498268 DNS prodajalce in kupce Večera poziva, da menjavo lastništva izpeljejo transparentno

Red Tree Heritage: Naravno okolje najbolj učinkovito varuje lokalno prebivalstvo
----------------------------
0.7060879980671629 Talibani sprožili ofenzivo v provinci Badgis, vladne enote so se umaknile

Red Tree Heritage: Naravno okolje najbolj učinkovito varuje lokalno prebivalstvo
----------------------------
0.70621186331355 Okoljski ministri pred veliko konferenco usklajujejo podnebne ambicije

Red Tree Heritage: Naravno okolje najbolj učinkovito varuje lokalno prebivalstvo
----------------------------
0.7111196912916157 SD: Ne pristajamo na ogrožanje prihodnosti naslednjih generacij

Red Tree

KeyboardInterrupt: 

In [50]:
print(pairwise_similarity.toarray())

[[1.         1.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.01592813 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.03359466 0.        ]
 [0.         0.         0.01592813 ... 0.03359466 1.         0.01236969]
 [0.         0.         0.         ... 0.         0.01236969 1.        ]]


In [41]:
print(tfidf)

  (0, 611)	0.160941192619832
  (0, 334)	0.160941192619832
  (0, 660)	0.11886353199183484
  (0, 426)	0.160941192619832
  (0, 242)	0.14859960658310292
  (0, 683)	0.160941192619832
  (0, 453)	0.14859960658310292
  (0, 875)	0.160941192619832
  (0, 709)	0.14859960658310292
  (0, 822)	0.160941192619832
  (0, 746)	0.160941192619832
  (0, 806)	0.160941192619832
  (0, 805)	0.160941192619832
  (0, 560)	0.160941192619832
  (0, 799)	0.10520183911044914
  (0, 7)	0.21040367822089828
  (0, 86)	0.21040367822089828
  (0, 237)	0.321882385239664
  (0, 212)	0.160941192619832
  (0, 704)	0.482823577859496
  (0, 534)	0.160941192619832
  (0, 272)	0.2624102360571278
  (0, 829)	0.321882385239664
  (1, 611)	0.160941192619832
  (1, 334)	0.160941192619832
  :	:
  (45, 686)	0.15142223920738376
  (45, 655)	0.15142223920738376
  (45, 370)	0.15142223920738376
  (45, 622)	0.15142223920738376
  (45, 720)	0.15142223920738376
  (45, 167)	0.15142223920738376
  (45, 140)	0.15142223920738376
  (45, 441)	0.15142223920738376
 